<a href="https://colab.research.google.com/github/LHwang268/Bicycle-Rental-Demand-Forecast/blob/main/GradientBoosting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install, import thư viện liên quan

tensorflow_data_validation với seaborn dùng để visualize dữ liệu (bên dưới), nhưng nặng quá nên mình chỉ xem thế thôi

In [ ]:
!pip install tensorflow_data_validation

In [ ]:
#!pip install -q seaborn

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

In [ ]:
import tensorflow_data_validation as tfdv
#import seaborn as sns

# Import dữ liệu

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving trainDataset.csv to trainDataset (1).csv


In [ ]:
import io
df = pd.read_csv(io.BytesIO(uploaded['trainDataset.csv']))

In [ ]:
df.head()

,id,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
0,0,2,1,4,2,0,3,1,2,0.50,0.4848,0.42,0.1940,9
1,1,4,0,9,12,0,1,1,2,0.68,0.6364,0.79,0.1642,209
2,2,2,1,4,0,0,3,1,1,0.34,0.3030,0.36,0.4179,30
3,3,2,1,6,15,0,0,0,1,0.70,0.6364,0.34,0.0000,644
4,4,4,1,10,10,0,3,1,1,0.56,0.5303,0.52,0.2985,217


In [ ]:
#df.describe().T

In [ ]:
train_stats = tfdv.generate_statistics_from_dataframe(df[['temp', 'atemp', 'hum', 'windspeed', 'cnt']])
tfdv.visualize_statistics(train_stats)

In [ ]:
#sns.pairplot(train_df[['workingday', 'temp', 'atemp', 'hum', 'windspeed', 'cnt']], diag_kind='kde')

In [ ]:
#sns.pairplot(df[['season', 'atemp', 'hum', 'windspeed']])

# Preprocessing

In [ ]:
# phân phối cnt bị lệch -> đưa về hàm log để chuẩn hóa
df['cnt_log'] = np.log(df['cnt'] + 1)

In [ ]:
train_len = int(len(df) * 0.8)
eval_len = len(df) - train_len

gbm_cols = [
    'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
    'holiday', 'workingday', 'season',
    'hr', 'weekday', 'yr'
    ]

train_df = df.iloc[:train_len].sample(frac=1, random_state=48).reset_index(drop=True)
eval_df = df.iloc[train_len:].sample(frac=1, random_state=48).reset_index(drop=True)

X_train = train_df[gbm_cols]
y_train = train_df['cnt_log']

X_val = eval_df[gbm_cols]
y_val = eval_df['cnt_log']

# Dùng Gradient Boosting

In [ ]:
params = {'n_estimators': 150, 'max_depth': 5, 'random_state': 0, 'min_samples_leaf' : 10, 'learning_rate': 0.1, 'subsample': 0.7, 'loss': 'ls'}
gbm_model = GradientBoostingRegressor(**params)


In [ ]:
model_cnt = gbm_model.fit(X_train, y_train)
y_pred = np.exp(model_cnt.predict(X_val)) - 1

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:290: FutureWarning: The loss 'ls' was deprecated in v1.0 and will be removed in version 1.2. Use 'squared_error' which is equivalent.
  FutureWarning,


In [ ]:
eval_df['pred'] = y_pred

In [ ]:
accuracy = model_cnt.score(X_val, y_val)
accuracy

0.9518560606631106

In [ ]:
((eval_df['pred'] - eval_df['cnt'])**2).mean()

1952.3349059298466

# Dùng Random Forest

In [ ]:
randomForest = RandomForestRegressor()
model2 = randomForest.fit(X_train, y_train)
y_pred = np.exp(model2.predict(X_val)) - 1

In [ ]:
eval_df['pred2'] = y_pred

In [ ]:
accuracy = model2.score(X_val, y_val)
accuracy

0.9468997015503152

In [ ]:
((eval_df['pred2'] - eval_df['cnt'])**2).mean()

2047.1153837448064

=> Thấy là Gradient Boosting ra kết quả tốt hơn (MSE nhỏ hơn)

# Dự báo và lưu kết quả

In [ ]:
uploaded1 = files.upload()

Saving submissionDataset.csv to submissionDataset.csv


In [ ]:
df_test = pd.read_csv(io.BytesIO(uploaded1['submissionDataset.csv']))

In [ ]:
X_test = df_test[gbm_cols]

In [ ]:
X_test.shape

(5214, 11)

In [ ]:
X_test.head()

,weathersit,temp,atemp,hum,windspeed,holiday,workingday,season,hr,weekday,yr
0,1,0.80,0.6970,0.27,0.1940,0,0,3,19,6,1
1,1,0.24,0.2273,0.41,0.2239,1,0,1,20,1,1
2,1,0.32,0.3030,0.66,0.2836,0,1,4,2,5,0
3,1,0.78,0.7121,0.52,0.3582,0,1,2,19,2,1
4,1,0.26,0.2273,0.56,0.3881,0,1,1,0,4,0


In [ ]:
test_predictions = np.exp(model_cnt.predict(X_test)) - 1

In [ ]:
test_predictions

array([441.63333019,  74.70604836,   9.94391844, ..., 552.20707142,
       293.30051289,  78.49472535])

In [ ]:
df_test['predict_cnt'] = test_predictions

In [ ]:
df_test

,id,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt,predict_cnt
0,0,3,1,6,19,0,6,0,1,0.80,0.6970,0.27,0.1940,NaN,441.633330
1,1,1,1,1,20,1,1,0,1,0.24,0.2273,0.41,0.2239,NaN,74.706048
2,2,4,0,10,2,0,5,1,1,0.32,0.3030,0.66,0.2836,NaN,9.943918
3,3,2,1,5,19,0,2,1,1,0.78,0.7121,0.52,0.3582,NaN,531.199628
4,4,1,0,1,0,0,4,1,1,0.26,0.2273,0.56,0.3881,NaN,10.433845
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5209,5209,2,1,6,10,0,3,1,1,0.82,0.7727,0.52,0.1343,NaN,181.890419
5210,5210,2,1,4,9,0,6,0,2,0.38,0.3939,0.37,0.0000,NaN,213.415907
5211,5211,3,1,6,12,0,0,0,1,0.80,0.6970,0.33,0.2239,NaN,552.207071
5212,5212,4,1,12,12,0,2,1,1,0.52,0.5000,0.68,0.1940,NaN,293.300513


In [ ]:
result = df_test[['id', 'predict_cnt']]

In [ ]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [ ]:
result.to_csv('result_lgbm.csv')
!cp result_lgbm.csv "drive/My Drive/"